In [31]:
import PyPDF2
import re

pdf_path = './PhonePe_Statement_Dec2023_Mar2024.pdf'

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)
        
        all_page_text = []
        for page_num in range(num_pages):
            # Get the page
            page = pdf_reader.pages[page_num]
            
            # Extract text from the page
            page_text = page.extract_text()
            clean_text = re.sub(r'[^\x00-\x7F]+', '', page_text)
            
            # Append the text to the list
            all_page_text.append(clean_text)
            
        return all_page_text

all_text = extract_text_from_pdf(pdf_path)

In [32]:
pages = []
for i in all_text:
    li = i.split('\n')
    li2 = li[1::2]
    pages.append(li2)

print(len(pages))

53


In [54]:
import re

transactions_text = []
# Regular expression pattern to match from month name up to "Transaction ID"
pattern = r"(\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\b.*?)(?=Transaction\sID)"

# Iterate over each page in the list of pages
for page in pages:
    # Join the lines of the current page into a single string
    page_text = " ".join(page)
    
    # Find all matches for the pattern in the current page
    matches = re.findall(pattern, page_text)
    transactions_text.append(matches)

text = transactions_text[0][0]
text =text.split(" ")
start = text.index("Amount")
text = " ".join(text[start+1:])
transactions_text[0][0] = text

transactions_nice = []
# Iterate over each transaction text in the transactions_text list
for transaction_page in transactions_text:
    for transaction in transaction_page:
        # Split the transaction string by spaces and strip each substring
        stripped_transaction = transaction.split(" ")
        cleared_transaction = [substr.strip() for substr in stripped_transaction if substr.strip()]
        
        # Process each cleared transaction
        data = {
            "month": cleared_transaction[0],
            "date": cleared_transaction[1],
            "year": cleared_transaction[2],
            "time": cleared_transaction[3] + " " + cleared_transaction[4],
            "type": cleared_transaction[5],
            "amount": cleared_transaction[6],
            "payee": " ".join(cleared_transaction[9:])
        }
        transactions_nice.append(data)

In [56]:
import pandas as pd

# Your code to process transactions_text and generate transactions_nice goes here...

# Convert transactions_nice to a pandas DataFrame
df = pd.DataFrame(transactions_nice)

df

,month,date,year,time,type,amount,payee
0,Mar,"31,",2024,09:17 PM,DEBIT,110,Vanshika kryana store
1,Mar,"31,",2024,05:15 PM,DEBIT,140,KALAVA GANESH
2,Mar,"30,",2024,07:49 PM,DEBIT,60,Vanshika kryana store
3,Mar,"30,",2024,07:45 PM,DEBIT,40,Mr. Om Prakash
4,Mar,"30,",2024,11:26 AM,DEBIT,300,MUNISH
...,...,...,...,...,...,...,...
410,Dec,"03,",2023,07:05 PM,DEBIT,435,S S General Store
411,Dec,"02,",2023,07:20 PM,DEBIT,140,Laundry
412,Dec,"01,",2023,11:31 PM,DEBIT,4,AMIRUL HAQUE
413,Dec,"01,",2023,11:31 PM,DEBIT,20,AMIRUL HAQUE


In [57]:
df.to_csv("transactions.csv"  , index=False)